# KE5006 Applied Research

### Identifying enhancers and their strength with deep neural networks

## Layer 2 Model with Physiochemical Property Features - Results

## Summary of Findings

## To be updated

* Validation Accuracy :
    * Ensemble of models of 2x16 GRU, 1x16 Dense
        * The best model ensemble of 5 models (around the best model) has an accuracy of 77.449%. The model ensemble beats the best single model accuracy of 76.735% from the same training run.
    * Ensemble of models of 1x16 conv1D, 2x16 GRU bidirectional, 1x8 Dense
        * The best model ensemble of 3 models (around the best model) has an accuracy of 76.429%. The model ensemble beats the best single model accuracy of 76.224% from the same training run.
* Test Accuracy :
    * Both ensembles achieved higher accuracy of **75.25%** and **75.50%** than the best single models of 74.00% and 73.75%. The single models were from training runs without warm restarts. 

## Load libraries

In [1]:
# Set the working directory (which contains the directories source, data, etc.)
import os
os.chdir(os.path.join(os.path.sep, 'home', 'tkokkeng', 'Documents', 'KE5006-AppliedResearch', 'enhancer'))
os.getcwd()

'/home/tkokkeng/Documents/KE5006-AppliedResearch/enhancer'

In [2]:
# Check if the directory containing the source files are in the path.
import sys
if os.path.join(os.getcwd(), 'source') not in sys.path:
    sys.path.append(os.path.join(os.getcwd(), 'source'))
sys.path

['/home/tkokkeng/python/python367/tsfvenv/lib/python36.zip',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/lib-dynload',
 '/usr/lib/python3.6',
 '',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/site-packages',
 '/home/tkokkeng/.local/lib/python3.6/site-packages',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/home/tkokkeng/python/python367/tsfvenv/lib/python3.6/site-packages/IPython/extensions',
 '/home/tkokkeng/.ipython',
 '/home/tkokkeng/Documents/KE5006-AppliedResearch/enhancer/source']

In [3]:
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import pickle

import myUtilities as mu

from sklearn.metrics import classification_report, confusion_matrix, matthews_corrcoef, make_scorer
from sklearn.metrics import recall_score, roc_auc_score, roc_curve, accuracy_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, load_model
from keras import layers
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

Using TensorFlow backend.


In [4]:
pd.set_option('precision', 3)

## Load data

In [5]:
strong_df = pd.read_csv(os.path.join('data', 'strong.csv'))
strong_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 2 columns):
id          742 non-null object
sequence    742 non-null object
dtypes: object(2)
memory usage: 11.7+ KB


In [6]:
strong_df['strong'] = np.ones((len(strong_df),))

In [7]:
strong_df.head()

,id,sequence,strong
0,CHRX_48897056_48897256,CACAATGTAGAAGCAGAGACACAGGAACCAGGCTTGGTGATGGCTC...,1.0
1,CHR12_6444339_6444539,GCCCTCACATTCCCTGGCCCATCCCCTCCACCTCAAAATTTACAAA...,1.0
2,CHR12_6444939_6445139,GAGCAGGAGGCCAGTCACCCTGAGTCAGCCACGGGGAGACGCTGCA...,1.0
3,CHR12_6445139_6445339,CCTCTGCTGAGAACAGGACTGGGGCTTCCAGGGCAACAGGAAGGGT...,1.0
4,CHR12_6445339_6445539,ACAGCCTTAAAGGGAGCTTTTCAGGGACCTCTGGCCAGTGGGGGAT...,1.0


In [8]:
weak_df = pd.read_csv(os.path.join('data', 'weak.csv'))
weak_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 742 entries, 0 to 741
Data columns (total 2 columns):
id          742 non-null object
sequence    742 non-null object
dtypes: object(2)
memory usage: 11.7+ KB


In [9]:
weak_df['strong'] = np.zeros((len(weak_df),))

In [10]:
weak_df.head()

,id,sequence,strong
0,CHR2_87930085_87930285,AAACAAACAAAAAAGAAGGTAATCAGTGTGGATGAGGATGTGGAAA...,0.0
1,CHR2_88301085_88301285,ATTCAAAATAAAATAATGAGTACACGTTTTGGAATAACGTCCTATT...,0.0
2,CHR3_126780310_126780510,ACATCCTTGCCAGCATTTGGTGTTTTTTTTTTTTTAATTAGCATAC...,0.0
3,CHR3_171979306_171979506,TGGCTTTTATTTTAAATTTAATTTGCTCAACCAGACACACAGTGTA...,0.0
4,CHR3_176244106_176244306,GCCTCTTTCTTGGATCAAGTTCATGATGCCAGATACTTTTCTCTTT...,0.0


## Combine the data frames to form a single dataset

In [11]:
all_data_df = pd.concat([strong_df, weak_df])
all_data_df.reset_index(drop=True, inplace=True)
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1484 entries, 0 to 1483
Data columns (total 3 columns):
id          1484 non-null object
sequence    1484 non-null object
strong      1484 non-null float64
dtypes: float64(1), object(2)
memory usage: 34.9+ KB


In [12]:
all_data_df.head()

,id,sequence,strong
0,CHRX_48897056_48897256,CACAATGTAGAAGCAGAGACACAGGAACCAGGCTTGGTGATGGCTC...,1.0
1,CHR12_6444339_6444539,GCCCTCACATTCCCTGGCCCATCCCCTCCACCTCAAAATTTACAAA...,1.0
2,CHR12_6444939_6445139,GAGCAGGAGGCCAGTCACCCTGAGTCAGCCACGGGGAGACGCTGCA...,1.0
3,CHR12_6445139_6445339,CCTCTGCTGAGAACAGGACTGGGGCTTCCAGGGCAACAGGAAGGGT...,1.0
4,CHR12_6445339_6445539,ACAGCCTTAAAGGGAGCTTTTCAGGGACCTCTGGCCAGTGGGGGAT...,1.0


All the sequences are of length 200 characters.

In [13]:
all_data_df['sequence'].map(lambda x: len(x)).value_counts()

200    1484
Name: sequence, dtype: int64

## Load the physiochemical property data

In [14]:
pcp_df = pd.read_csv(os.path.join('data', 'S2.csv'), index_col=0)
pcp_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16 entries, AA to TT
Data columns (total 6 columns):
Rise     16 non-null float64
Roll     16 non-null float64
Shift    16 non-null float64
Slide    16 non-null float64
Tilt     16 non-null float64
Twist    16 non-null float64
dtypes: float64(6)
memory usage: 896.0+ bytes


In [15]:
scaler = MinMaxScaler()
pcp_df.loc[:, :] = scaler.fit_transform(pcp_df.values)
pcp_df

,Rise,Roll,Shift,Slide,Tilt,Twist
AA,0.430,0.403,1.000,0.545,0.4,0.833
AC,0.818,0.696,0.619,1.000,0.7,0.833
AG,0.258,0.316,0.763,0.773,0.3,0.792
AT,0.861,1.000,0.320,0.864,0.6,0.750
CA,0.045,0.221,0.361,0.091,0.1,0.292
CC,0.548,0.171,0.732,0.545,0.3,1.000
CG,0.000,0.304,0.371,0.000,0.0,0.333
CT,0.258,0.316,0.763,0.773,0.3,0.792
GA,0.706,0.278,0.619,0.500,0.4,0.833
GC,1.000,0.536,0.495,0.500,1.0,0.750


## Prepare the sequence data for modelling

Create a transformation pipleline to prepare the training dataset for RNN.

In [16]:
# This class selects the desired attributes and drops the rest.
class DataFrameSelector(BaseEstimator, TransformerMixin):

    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.attribute_names]

In [17]:
# This class converts a nucleotide base (A, C, G, T) to one-hot-encoding.
class one_hot_encoder(BaseEstimator, TransformerMixin):

    def __init__(self):
        self.tokenizer = Tokenizer(num_words=4, lower=False, char_level=True)

    def fit(self, X, y=None):
        # Note that X is a data frame.
        # Fit the tokenizer on the 1st sequence in the dataset.
        self.tokenizer.fit_on_texts(X.iloc[0, 0])
        self.len_sequence = len(X.iloc[0, 0])
        return self

    def transform(self, X):
        # Note that X is a data frame.
        one_hot_X = X.iloc[:, 0].map(lambda x: self.tokenizer.texts_to_matrix(x, mode='binary')).values
        one_hot_X = np.concatenate(one_hot_X)
        one_hot_X = np.reshape(one_hot_X, (-1, self.len_sequence, 4))
        return one_hot_X

In [18]:
# This class converts a sequence of nucleotide bases (A, C, G, T) to a sequence of dinucleotides and then to a sequence of pysiochemical properties of each dinucleotide.
class pcp_encoder(BaseEstimator, TransformerMixin):

    def __init__(self, pcp_df):
        self.pcp_df = pcp_df

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Note that X is a data frame.
        dinuc_seq = X.iloc[:, 0].map(lambda x: [ x[i:i+2] for i in range(len(x) - 1) ])
        pcp_seq = dinuc_seq.map(lambda x: [ pcp_df[j][i] for i in x for j in pcp_df.columns.tolist() ])
        # Pad with -1 for last element of sequence; it does not have an associated di-nucleotide
        pcp_seq = pcp_seq.map(lambda x: np.array(x + [-1. for i in range(len(pcp_df.columns))]).reshape((len(X.iloc[0, 0]), len(pcp_df.columns)))).values
        # pandas values returns a 1-D array of objects; use numpy stack to reshape it to a multi-dimensional array
        return np.stack(pcp_seq)

In [19]:
# This class shapes a numpy array.
class Array_Shaper(BaseEstimator, TransformerMixin):
    
    def __init__(self, shape):
        self.shape = shape
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.reshape(self.shape)

In [20]:
attrbs = ['sequence']
num_bases = 4  # number of nucleotide bases
num_pcp = 6  # number of di-nucleotide physiochemical properties
len_seq = len(all_data_df['sequence'][0])
one_hot_pipeline = Pipeline([
    ('selector', DataFrameSelector(attrbs)),
    ('one_hot_encoder', one_hot_encoder()),
    ('array_shaper2D', Array_Shaper((-1, num_bases)))
])
pcp_pipeline = Pipeline([
    ('selector', DataFrameSelector(attrbs)),
    ('pcp_encoder', pcp_encoder(pcp_df)),
    ('array_shaper2D', Array_Shaper((-1, num_pcp)))
])
union_pipeline = FeatureUnion(transformer_list=[
    ("one_hot_pipeline", one_hot_pipeline),
    ("pcp_pipeline", pcp_pipeline)
])
my_pipeline = Pipeline([
    ('feature_combiner', union_pipeline),
    ('array_shaper3D', Array_Shaper((-1, len_seq, num_bases + num_pcp)))
])

In [21]:
X = my_pipeline.fit_transform(all_data_df)
X.shape

(1484, 200, 10)

Check the 1st sequence is correctly encoded.

In [22]:
X[0, :10, :]

array([[0.        , 0.        , 0.        , 0.        , 0.04545455,
        0.22053232, 0.36082474, 0.09090909, 0.1       , 0.29166667],
       [0.        , 0.        , 0.        , 1.        , 0.81818182,
        0.69581749, 0.6185567 , 1.        , 0.7       , 0.83333333],
       [0.        , 0.        , 0.        , 0.        , 0.04545455,
        0.22053232, 0.36082474, 0.09090909, 0.1       , 0.29166667],
       [0.        , 0.        , 0.        , 1.        , 0.43030303,
        0.40304183, 1.        , 0.54545455, 0.4       , 0.83333333],
       [0.        , 0.        , 0.        , 1.        , 0.86060606,
        1.        , 0.31958763, 0.86363636, 0.6       , 0.75      ],
       [0.        , 0.        , 1.        , 0.        , 0.04545455,
        0.22053232, 0.36082474, 0.09090909, 0.1       , 0.29166667],
       [0.        , 1.        , 0.        , 0.        , 0.81818182,
        0.69581749, 0.6185567 , 1.        , 0.7       , 0.83333333],
       [0.        , 0.        , 1.       

In [23]:
X[0, -10:, :]

array([[ 0.        ,  0.        ,  1.        ,  0.        ,  0.43030303,
         0.40304183,  1.        ,  0.54545455,  0.4       ,  0.83333333],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.04545455,
         0.22053232,  0.36082474,  0.09090909,  0.1       ,  0.29166667],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.81818182,
         0.69581749,  0.6185567 ,  1.        ,  0.7       ,  0.83333333],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.70606061,
         0.27756654,  0.6185567 ,  0.5       ,  0.4       ,  0.83333333],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.54848485,
         0.17110266,  0.73195876,  0.54545455,  0.3       ,  1.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.25757576,
         0.31558935,  0.7628866 ,  0.77272727,  0.3       ,  0.79166667],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.13636364

In [24]:
y = all_data_df['strong'].values
y.shape

(1484,)

In [25]:
y[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

## Split the dataset into train / validation sets

For the initial base model, we will use a simple train / validation split. 5-fold cross-validation will be used during model fine-tuning to obtain the final model.

In [26]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)

In [27]:
X_train.shape

(1187, 200, 10)

In [28]:
X_train[0][:10]

array([[0.        , 0.        , 1.        , 0.        , 0.70606061,
        0.27756654, 0.6185567 , 0.5       , 0.4       , 0.83333333],
       [0.        , 0.        , 0.        , 0.        , 0.25757576,
        0.31558935, 0.7628866 , 0.77272727, 0.3       , 0.79166667],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.13636364, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.25757576,
        0.31558935, 0.7628866 , 0.77272727, 0.3       , 0.79166667],
       [0.        , 1.        , 0.        , 0.        , 0.70606061,
        0.27756654, 0.6185567 , 0.5       , 0.4       , 0.83333333],
       [0.        , 0.        , 0.        , 1.        , 0.81818182,
        0.69581749, 0.6185567 , 1.        , 0.7       , 0.83333333],
       [0.        , 0.        , 0.        , 0.        , 0.25757576,
        0.31558935, 0.7628866 , 0.77272727, 0.3       , 0.79166667],
       [0.        , 0.        , 1.       

In [29]:
y_train.shape

(1187,)

In [30]:
y_train[0]

1.0

## Load models

### Ensemble of models of 2x16 GRU, 1x16 Dense
* Dropouts (.1/.1/.1)
* Warm restarts cycle = 200, lr=0.003
* Saved models are best in each cycle

In [31]:
from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
set_random_seed(456)

In [32]:
# Best model
model = load_model(os.path.join('models-layer2',
                                'layer2-2x16gru1x16dense-dropout010101-wr01.best-epch228.h5'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, None, 16)          1344      
_________________________________________________________________
gru_2 (GRU)                  (None, 16)                1632      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 3,265
Trainable params: 3,265
Non-trainable params: 0
_________________________________________________________________


In [33]:
results_df = pd.DataFrame()

In [34]:
results_df['best'] = pd.Series(model.predict_classes(X_val, batch_size=128, verbose=1).flatten())

297/297 [==============================] - 1s 3ms/step


In [35]:
best_acc = 100. * (len(y_val) - (np.abs(results_df['best'].values - y_val)).sum()) / len(y_val)
'{:06.3f}'.format(best_acc)

'63.300'

The best model from training was saved separately from the models saved in each of the cycles below. It will be the same as one of the best models from the cycles. The best model from the first cycle (1-200 epoch) is not used as the loss had not converged to a minimum.

In [36]:
# Path to the weights saved for the best model in each cycle
path = [os.path.join('models-layer2', 'layer2-2x16gru1x16dense-dropout010101-wr01-weights', i)
        for i in ['model_wgts_cyc0400.h5', 'model_wgts_cyc0600.h5', 'model_wgts_cyc0800.h5', 'model_wgts_cyc1000.h5', 'model_wgts_cyc1200.h5', 'model_wgts_cyc1400.h5', 'model_wgts_cyc1600.h5',
                 'model_wgts_cyc1800.h5', 'model_wgts_cyc2000.h5', 'model_wgts_cyc2200.h5', 'model_wgts_cyc2400.h5', 'model_wgts_cyc2600.h5', 'model_wgts_cyc2800.h5', 'model_wgts_cyc3000.h5']]
path

['models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc0400.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc0600.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc0800.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc1000.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc1200.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc1400.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc1600.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc1800.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc2000.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc2200.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc2400.h5',

In [37]:
# Calculate the predictions for the best models in each cycle.
for idx, a_file in enumerate(path):
    model.load_weights(filepath=a_file, by_name=False)
    results_df['model' + str(idx)] = model.predict_classes(X_val, batch_size=128, verbose=1)

297/297 [==============================] - 0s 633us/step


In [38]:
# Compare the predictions of each cycle's best model with the overall best model.
# Count the number of different predictions.
for i in range(14):
    print((results_df['best'] - results_df['model' + str(i)]).abs().sum())

0
69
41
112
68
97
113
105
102
98
104
109
107
104


In [39]:
# calculate the accuracy for each model
for i in range(14):
    acc = 100. * (len(y_val) - (np.abs(results_df['model' + str(i)].values - y_val)).sum()) / len(y_val)
    print('model' + str(i) + ' = {:06.3f}%'.format(acc))

model0 = 63.300%
model1 = 62.963%
model2 = 62.290%
model3 = 59.259%
model4 = 59.933%
model5 = 59.596%
model6 = 60.943%
model7 = 60.269%
model8 = 60.606%
model9 = 59.259%
model10 = 58.586%
model11 = 59.596%
model12 = 59.596%
model13 = 59.933%


#### Evalute the Model Ensembles
#### Use models 0-12 (13 models)

In [40]:
acc_list = []

In [41]:
cols = results_df.columns.tolist()
cols.remove('best')
cols.remove('model13')

# Decision threshold for majority voting
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))

threshold = 6


In [42]:
# Calculate the ensemble prediction
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

,best,model0,model1,model2,model3,model4,model5,model6,model7,model8,model9,model10,model11,model12,model13,ensemble
0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [43]:
# Calculate the model ensemble accuracy 
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('0-12', 13, acc))

model ensemble accuracy = 62.290%


#### Use models 0-10 (11 models)

In [44]:
cols = results_df.columns.tolist()
for i in ['best', 'model11', 'model12', 'model13', 'ensemble']:
    cols.remove(i)
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

threshold = 5


,best,model0,model1,model2,model3,model4,model5,model6,model7,model8,model9,model10,model11,model12,model13,ensemble
0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [45]:
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('0-10', 11, acc))

model ensemble accuracy = 62.290%


#### Use models 0-8 (9 models)

In [46]:
cols = results_df.columns.tolist()
for i in ['best', 'model9', 'model10', 'model11', 'model12', 'model13', 'ensemble']:
    cols.remove(i)
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

threshold = 4


,best,model0,model1,model2,model3,model4,model5,model6,model7,model8,model9,model10,model11,model12,model13,ensemble
0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [47]:
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('0-8', 9, acc))

model ensemble accuracy = 60.606%


#### Use models 0-6 (7 models)

In [48]:
cols = results_df.columns.tolist()
for i in ['best', 'model7', 'model8', 'model9', 'model10', 'model11', 'model12', 'model13', 'ensemble']:
    cols.remove(i)
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

threshold = 3


,best,model0,model1,model2,model3,model4,model5,model6,model7,model8,model9,model10,model11,model12,model13,ensemble
0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [49]:
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('0-6', 7, acc))

model ensemble accuracy = 59.259%


#### Use models 0-4 (5 models)

In [50]:
cols = results_df.columns.tolist()
for i in ['best', 'model5', 'model6', 'model7', 'model8', 'model9', 'model10', 'model11', 'model12', 'model13', 'ensemble']:
    cols.remove(i)
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

threshold = 2


,best,model0,model1,model2,model3,model4,model5,model6,model7,model8,model9,model10,model11,model12,model13,ensemble
0,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0


In [51]:
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('0-4', 5, acc))

model ensemble accuracy = 61.279%


#### Use models 0, 1, 2 (3 models)

In [52]:
cols = ['model0', 'model1', 'model2']
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('0, 1, 2', 3, acc))

threshold = 1
model ensemble accuracy = 63.636%


In [53]:
acc_df = pd.DataFrame(acc_list, columns=['Models', 'Number of models', 'Accuracy'])
acc_df

,Models,Number of models,Accuracy
0,0-12,13,62.290
1,0-10,11,62.290
2,0-8,9,60.606
3,0-6,7,59.259
4,0-4,5,61.279
5,"0, 1, 2",3,63.636


**Model ensemble 5** has the best accuracy. The model ensemble beats the best single model accuracy of 63.30%.

### Ensemble of models of 1x16 conv1D, 2x16 GRU bidirectional, 1x8 Dense
* Dropouts (.6/.6/.6)
* Warm restarts cycle = 200, max_lr=.001
* Saved model are best in each cycle

In [54]:
from numpy.random import seed
seed(123)
from tensorflow import set_random_seed
set_random_seed(456)

In [55]:
# Best model
model = load_model(os.path.join('models-layer2', 'layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01.best-epch1224.h5'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 192, 16)           1456      
_________________________________________________________________
batch_normalization_1 (Batch (None, 192, 16)           64        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 96, 16)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 96, 32)            3264      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32)                4800      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
__________

In [56]:
results_df = pd.DataFrame()

In [57]:
results_df['best'] = pd.Series(model.predict_classes(X_val, batch_size=128, verbose=1).flatten())

297/297 [==============================] - 2s 7ms/step


In [58]:
best_acc = 100. * (len(y_val) - (np.abs(results_df['best'].values - y_val)).sum()) / len(y_val)
'{:06.3f}%'.format(best_acc)

'59.596%'

In [59]:
path = [os.path.join('models-layer2', 'layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights', i)
        for i in ['model_wgts_cyc0200.h5', 'model_wgts_cyc0400.h5', 'model_wgts_cyc0600.h5',
                  'model_wgts_cyc0800.h5', 'model_wgts_cyc1000.h5', 'model_wgts_cyc1200.h5',
                  'model_wgts_cyc1400.h5', 'model_wgts_cyc1600.h5',
                  'model_wgts_cyc1800.h5', 'model_wgts_cyc2000.h5']]
path

['models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc0200.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc0400.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc0600.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc0800.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc1000.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc1200.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc1400.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc1600.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc1800.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc2000.h5']

In [60]:
for idx, a_file in enumerate(path):
    model.load_weights(filepath=a_file, by_name=False)
    results_df['model' + str(idx)] = model.predict_classes(X_val, batch_size=128, verbose=1)

297/297 [==============================] - 0s 1ms/step


In [61]:
for i in range(10):
    print((results_df['best'] - results_df['model' + str(i)]).abs().sum())

120
91
86
55
36
24
0
19
24
23


In [62]:
for i in range(10):
    acc = 100. * (len(y_val) - (np.abs(results_df['model' + str(i)].values - y_val)).sum()) / len(y_val)
    print('model' + str(i) + ' = {:06.3f}%'.format(acc))

model0 = 54.209%
model1 = 57.912%
model2 = 57.576%
model3 = 57.239%
model4 = 57.576%
model5 = 58.923%
model6 = 59.596%
model7 = 58.586%
model8 = 57.576%
model9 = 57.239%


In [63]:
acc_list = []

#### Use models 1-9 (9 models)

In [64]:
cols = results_df.columns.tolist()
cols.remove('best')
cols.remove('model0')
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

threshold = 4


,best,model0,model1,model2,model3,model4,model5,model6,model7,model8,model9,ensemble
0,1,0,1,1,1,1,1,1,1,1,1,1
1,1,0,1,1,1,1,0,1,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,0,0,0,0,0,0,0,0


In [65]:
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('1-9', 9, acc))

model ensemble accuracy = 58.249%


#### Use models 3-9 (7 models)

In [66]:
cols = results_df.columns.tolist()
for i in ['best', 'model0', 'model1', 'model2', 'ensemble']:
    cols.remove(i)
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

threshold = 3


,best,model0,model1,model2,model3,model4,model5,model6,model7,model8,model9,ensemble
0,1,0,1,1,1,1,1,1,1,1,1,1
1,1,0,1,1,1,1,0,1,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,0,0,0,0,0,0,0,0


In [67]:
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('3-9', 7, acc))

model ensemble accuracy = 58.249%


#### Use models 4-8 (5 models)

In [68]:
cols = results_df.columns.tolist()
for i in ['best', 'model0', 'model1', 'model2', 'model3', 'model9', 'ensemble']:
    cols.remove(i)
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

threshold = 2


,best,model0,model1,model2,model3,model4,model5,model6,model7,model8,model9,ensemble
0,1,0,1,1,1,1,1,1,1,1,1,1
1,1,0,1,1,1,1,0,1,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,0,0,0,0,0,0,0,0


In [69]:
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('4-8', 5, acc))

model ensemble accuracy = 58.923%


#### Use models 5-7 (3 models)

In [70]:
cols = ['model5', 'model6', 'model7']
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

threshold = 1


,best,model0,model1,model2,model3,model4,model5,model6,model7,model8,model9,ensemble
0,1,0,1,1,1,1,1,1,1,1,1,1
1,1,0,1,1,1,1,0,1,1,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,1,0,0,0,0,0,0,0,0


In [71]:
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('5-7', 3, acc))

model ensemble accuracy = 59.596%


#### Use models 4, 6, 8 (3 models)

In [72]:
cols = ['model4', 'model6', 'model8']
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('4, 6, 8', 3, acc))

threshold = 1
model ensemble accuracy = 58.923%


#### Use models 3, 6, 9

In [73]:
cols = ['model3', 'model6', 'model9']
threshold = len(cols) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.loc[:, cols].apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
acc = 100. * (len(y_val) - (np.abs(results_df['ensemble'].values - y_val)).sum()) / len(y_val)
print('model ensemble accuracy = {:06.3f}%'.format(acc))
acc_list.append(('3, 6, 9', 3, acc))

threshold = 1
model ensemble accuracy = 58.586%


In [74]:
acc_df = pd.DataFrame(acc_list, columns=['Models', 'Number of models', 'Accuracy'])
acc_df

,Models,Number of models,Accuracy
0,1-9,9,58.249
1,3-9,7,58.249
2,4-8,5,58.923
3,5-7,3,59.596
4,"4, 6, 8",3,58.923
5,"3, 6, 9",3,58.586


**Model ensemble 3** has the best accuracy which is equivalent to the best model accuracy of 59.60%

 ## Evaluate model ensembles, best single models on the test set

In [75]:
test_df = pd.read_csv(os.path.join('data', 'independent.csv'))
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 3 columns):
id          400 non-null object
type        400 non-null object
sequence    400 non-null object
dtypes: object(3)
memory usage: 9.5+ KB


In [76]:
test_df.head()

,id,type,sequence
0,Chr11_6627824_6628024,strong,ATGCTGCCAGAAGGAAAAGGGGTGGAATTAATGAAACTGGAAGGTT...
1,Chr11_9587224_9587424,strong,GGCATTTTTTAACCTGTGTTTCATTTTCATCTGTGAAATGTGAATA...
2,Chr11_65187024_65187224,strong,GAAACCACAGAGCTGACCTGGCTTCAGAACAAGATGTGGGGCTCCA...
3,Chr10_74014594_74014794,strong,TTTGCATAGGGGCATTACCACTGGACTTGGGCTCAGAGCAAGTGTT...
4,Chr10_105667810_105668010,strong,CGGGAGGCGGGGGTTGCAGTGAGCCAAGATCACACCACTGCACTCC...


In [77]:
test_df['type'].value_counts()

non-enhancer    200
weak            100
strong          100
Name: type, dtype: int64

In [78]:
test_df['enhancer'] = test_df['type'].map(lambda x: 0.0 if x == 'non-enhancer' else 1.0)

In [79]:
test_df.head()

,id,type,sequence,enhancer
0,Chr11_6627824_6628024,strong,ATGCTGCCAGAAGGAAAAGGGGTGGAATTAATGAAACTGGAAGGTT...,1.0
1,Chr11_9587224_9587424,strong,GGCATTTTTTAACCTGTGTTTCATTTTCATCTGTGAAATGTGAATA...,1.0
2,Chr11_65187024_65187224,strong,GAAACCACAGAGCTGACCTGGCTTCAGAACAAGATGTGGGGCTCCA...,1.0
3,Chr10_74014594_74014794,strong,TTTGCATAGGGGCATTACCACTGGACTTGGGCTCAGAGCAAGTGTT...,1.0
4,Chr10_105667810_105668010,strong,CGGGAGGCGGGGGTTGCAGTGAGCCAAGATCACACCACTGCACTCC...,1.0


In [80]:
test_df.loc[test_df['type'] == 'weak', :][:5]

,id,type,sequence,enhancer
100,hg19_ct_UserTrack_3545_11005,weak,TTATGGTCACCTTCGACCCCAGAAATAATGGTCTCTGTTGTCAGAT...,1.0
101,hg19_ct_UserTrack_3545_8529,weak,CATCCAGGCTTGGTCCTGGTTGTTCCTTGCTGTTATACCAGCCTGG...,1.0
102,hg19_ct_UserTrack_3545_7245,weak,TTGTTTTTTTCTGTTTTGAGACGGAGTTTCGCTCTTGTTGCCCAGG...,1.0
103,hg19_ct_UserTrack_3545_12669,weak,ACTGTTAAATAGCAAAAATTATTGAGCTCAAACCATCTAACCAGGT...,1.0
104,hg19_ct_UserTrack_3545_5404,weak,GAGAATTAAGTTTGTATTAAGTTGGAGACCAGGGCAGATGGAAAGA...,1.0


In [81]:
test_df.loc[test_df['type'] == 'non-enhancer', :][:5]

,id,type,sequence,enhancer
200,hg19_ct_UserTrack_3545_158,non-enhancer,AATTTTCTCATTTTCTCATAAAGTTTAACAGTTGTTTATTTGAGTC...,0.0
201,hg19_ct_UserTrack_3545_57,non-enhancer,ACTGGTTATCTTTTAGGACTAGTTAATATAACCCATTCTCTAACCA...,0.0
202,hg19_ct_UserTrack_3545_762,non-enhancer,ATGCATATGTTCTTCAGTAAACAGAGCAGCCACTGGTACCACAGGA...,0.0
203,hg19_ct_UserTrack_3545_78,non-enhancer,CTGCTCTCCTCGCTCTATAAAAGTCAGAGTGCCTAAGCTGTTAATT...,0.0
204,hg19_ct_UserTrack_3545_9,non-enhancer,GCTTGGGTATATATTGTCCAATATAGCAGGCCTCATGTGCTCCTTA...,0.0


In [82]:
# X_test = my_pipeline.fit_transform(test_df)  # wrong!
X_test = my_pipeline.transform(test_df)
X_test.shape

(400, 200, 10)

In [83]:
X_test[0, :10, :]

array([[0.        , 0.        , 0.        , 1.        , 0.86060606,
        1.        , 0.31958763, 0.86363636, 0.6       , 0.75      ],
       [0.        , 0.        , 1.        , 0.        , 0.04545455,
        0.22053232, 0.36082474, 0.09090909, 0.1       , 0.29166667],
       [0.        , 1.        , 0.        , 0.        , 1.        ,
        0.53612167, 0.49484536, 0.5       , 1.        , 0.75      ],
       [0.        , 0.        , 0.        , 0.        , 0.25757576,
        0.31558935, 0.7628866 , 0.77272727, 0.3       , 0.79166667],
       [0.        , 0.        , 1.        , 0.        , 0.04545455,
        0.22053232, 0.36082474, 0.09090909, 0.1       , 0.29166667],
       [0.        , 1.        , 0.        , 0.        , 1.        ,
        0.53612167, 0.49484536, 0.5       , 1.        , 0.75      ],
       [0.        , 0.        , 0.        , 0.        , 0.54848485,
        0.17110266, 0.73195876, 0.54545455, 0.3       , 1.        ],
       [0.        , 0.        , 0.       

In [84]:
y_test = test_df['enhancer'].values
y_test.shape

(400,)

In [85]:
y_test[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

### Use the best ensemble layer 1 classifier predictions as Input for the Layer 2 Classifier

In [86]:
# Best model. Use this model to load the saved weights for the model ensemble.
model = load_model(os.path.join('models', 'pcp-2x16gru1x16dense-dropout010101-wr41.best-epch1283.h5'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, None, 16)          1344      
_________________________________________________________________
gru_2 (GRU)                  (None, 16)                1632      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 3,265
Trainable params: 3,265
Non-trainable params: 0
_________________________________________________________________


In [87]:
results_df = pd.DataFrame()

In [88]:
path = [os.path.join('models', 'pcp-2x16gru1x16dense-dropout010101-wr41-weights', i)
        for i in ['model_wgts_cyc1000.h5', 'model_wgts_cyc1200.h5', 'model_wgts_cyc1400.h5',
                  'model_wgts_cyc1600.h5', 'model_wgts_cyc1800.h5']]
path

['models/pcp-2x16gru1x16dense-dropout010101-wr41-weights/model_wgts_cyc1000.h5',
 'models/pcp-2x16gru1x16dense-dropout010101-wr41-weights/model_wgts_cyc1200.h5',
 'models/pcp-2x16gru1x16dense-dropout010101-wr41-weights/model_wgts_cyc1400.h5',
 'models/pcp-2x16gru1x16dense-dropout010101-wr41-weights/model_wgts_cyc1600.h5',
 'models/pcp-2x16gru1x16dense-dropout010101-wr41-weights/model_wgts_cyc1800.h5']

In [89]:
for idx, a_file in enumerate(path):
    model.load_weights(filepath=a_file, by_name=False)
    results_df['model' + str(idx)] = pd.Series(model.predict_classes(X_test, batch_size=128, verbose=1).flatten())

400/400 [==============================] - 0s 629us/step


In [90]:
threshold = len(results_df.columns) // 2
print('threshold = {}'.format(threshold))
test_df['layer1_pred'] = results_df.apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)

threshold = 2


In [91]:
# Count the number of false negatives in the layer 1 predictions
len(test_df.loc[(test_df['enhancer'] == 1.0) & (test_df['layer1_pred'] == 0.0), :]) 

54

In [92]:
# Get the correctly predicted samples.
layer2_df = test_df.loc[(test_df['enhancer'] == 1.0) & (test_df['layer1_pred'] == 1.0), :]
#X_layer2_test = my_pipeline.fit_transform(layer2_df)
X_layer2_test = my_pipeline.transform(layer2_df)
X_layer2_test.shape

(146, 200, 10)

In [93]:
layer2_df['type'].value_counts()

strong    89
weak      57
Name: type, dtype: int64

In [94]:
y_layer2_test = layer2_df['type'].map(lambda x: 1.0 if x == 'strong' else 0.0).values
y_layer2_test.shape

(146,)

### Parameters, Functions for Scoring

In [95]:
acc_list = []

In [96]:
# customer scorer function for specificity
def specificity(y, y_pred):
    # print(y_pred)
    tn, fp = 0, 0
    for idx, a_pred in enumerate(y_pred):
        if a_pred and not(y[idx]):
            fp += 1
        elif not(a_pred) and not(y[idx]):
            tn += 1
    return tn/(tn+fp)

In [97]:
# function for all scores
def all_scores(y, y_pred):
    acc = accuracy_score(y, y_pred) * 100.
    recall = recall_score(y, y_pred) * 100.
    spec = specificity(y, y_pred) * 100.
    auc = roc_auc_score(y, y_pred) * 100.
    mcc = matthews_corrcoef(y, y_pred)
    return (acc, mcc, recall, spec, auc)

### 2x16 GRU, 1x16 Dense Best Single Model
* Dropout 0.1
* Best accuracy @epoch 865

In [98]:
# Best model
model = load_model(os.path.join('models-layer2', 'layer2-2x16gru1x16dense-dropout0101.best-epch547.h5'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, None, 16)          1344      
_________________________________________________________________
gru_2 (GRU)                  (None, 16)                1632      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 3,265
Trainable params: 3,265
Non-trainable params: 0
_________________________________________________________________


In [99]:
y_pred = model.predict_classes(X_layer2_test, batch_size=None, verbose=1)

146/146 [==============================] - 1s 5ms/step


In [100]:
acc_list.append(('2x16GRU 1x16D', 'single') + all_scores(y_layer2_test, y_pred))

### 1x16 Conv1D, 2x16 GRU Bidirectional, 1x8 Dense Best Single Model
* Dropout 0.6
* Best accuracy @epoch 693

In [101]:
# Best model
model = load_model(os.path.join('models-layer2', 'layer2-1x16cv-2x16gruB-1x8d-dropout060606.best-epch2081.h5'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 192, 16)           1456      
_________________________________________________________________
batch_normalization_1 (Batch (None, 192, 16)           64        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 96, 16)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 96, 32)            3264      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32)                4800      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
__________

In [102]:
y_pred = model.predict_classes(X_layer2_test, batch_size=None, verbose=1)

146/146 [==============================] - 1s 7ms/step


In [103]:
acc_list.append(('1x16 Conv1D 2x16 GRU-B 1x8 Dense', 'single') + all_scores(y_layer2_test, y_pred))

### Model Ensemble of 2x16 GRU, 1x16 Dense using 3 models
* Dropouts 0.1
* Warm restarts cycle = 200, lr=0.003
* Using models 0-2 out of 0-10

In [104]:
# Best model. Use this model to load the saved weights for the model ensemble.
model = load_model(os.path.join('models-layer2', 'layer2-2x16gru1x16dense-dropout010101-wr01.best-epch228.h5'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, None, 16)          1344      
_________________________________________________________________
gru_2 (GRU)                  (None, 16)                1632      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 3,265
Trainable params: 3,265
Non-trainable params: 0
_________________________________________________________________


In [105]:
results_df = pd.DataFrame()

In [106]:
path = [os.path.join('models-layer2', 'layer2-2x16gru1x16dense-dropout010101-wr01-weights', i)
        for i in ['model_wgts_cyc0400.h5', 'model_wgts_cyc0600.h5', 'model_wgts_cyc0800.h5']]
path

['models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc0400.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc0600.h5',
 'models-layer2/layer2-2x16gru1x16dense-dropout010101-wr01-weights/model_wgts_cyc0800.h5']

In [107]:
for idx, a_file in enumerate(path):
    model.load_weights(filepath=a_file, by_name=False)
    results_df['model' + str(idx)] = pd.Series(model.predict_classes(X_layer2_test, batch_size=128, verbose=1).flatten())

146/146 [==============================] - 0s 875us/step


In [108]:
results_df.head()

,model0,model1,model2
0,0,1,1
1,1,1,1
2,1,1,1
3,0,1,1
4,0,1,1


In [109]:
threshold = len(results_df.columns) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

threshold = 1


,model0,model1,model2,ensemble
0,0,1,1,1
1,1,1,1,1
2,1,1,1,1
3,0,1,1,1
4,0,1,1,1


In [110]:
acc_list.append(('2x16GRU 1x16D', 'ensemble') + all_scores(y_layer2_test, results_df['ensemble'].values))

#### Model Ensemble for base learner 1x16cv, 2x16gruB, 1x8d, dropout 060606 using 3 models (1, 4 and 7) 

### Model Ensemble of 1x16 Conv1D, 2x16 GRU Bidirectional, 1x8 Dense using 3 models
* Dropouts (.6/.6/.6)
* Warm restarts cycle = 200, max_lr=.001
* Using models 5-7 out of 0-9

In [111]:
# Best model
model = load_model(os.path.join('models-layer2', 'layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01.best-epch1224.h5'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 192, 16)           1456      
_________________________________________________________________
batch_normalization_1 (Batch (None, 192, 16)           64        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 96, 16)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 96, 32)            3264      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32)                4800      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 264       
__________

In [112]:
results_df = pd.DataFrame()

In [113]:
path = [os.path.join('models-layer2', 'layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights', i)
        for i in ['model_wgts_cyc1200.h5', 'model_wgts_cyc1400.h5', 'model_wgts_cyc1600.h5']]
path

['models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc1200.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc1400.h5',
 'models-layer2/layer2-1x16cv-2x16gruB-1x8d-dropout060606-wr01-weights/model_wgts_cyc1600.h5']

In [114]:
for idx, a_file in enumerate(path):
    model.load_weights(filepath=a_file, by_name=False)
    results_df['model' + str(idx)] = pd.Series(model.predict_classes(X_layer2_test, batch_size=128, verbose=1).flatten())

146/146 [==============================] - 0s 1ms/step


In [115]:
results_df.head()

,model0,model1,model2
0,1,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,0,0,0


In [116]:
threshold = len(results_df.columns) // 2
print('threshold = {}'.format(threshold))
results_df['ensemble'] = results_df.apply(lambda x: 1 if x.sum() > threshold else 0, axis=1)
results_df.head()

threshold = 1


,model0,model1,model2,ensemble
0,1,1,1,1
1,1,1,1,1
2,1,1,1,1
3,1,1,1,1
4,0,0,0,0


In [117]:
acc_list.append(('1x16 Conv1D 2x16 GRU-B 1x8 Dense', 'ensemble')
                + all_scores(y_layer2_test, results_df['ensemble'].values))

In [118]:
acc_df = pd.DataFrame(acc_list, columns=['Model', 'Type', 'Accuracy(%)', 'MCC', 'Recall(%)', 'Specificity(%)',
                                         'AUC(%)'])
acc_df

,Model,Type,Accuracy(%),MCC,Recall(%),Specificity(%),AUC(%)
0,2x16GRU 1x16D,single,60.959,0.100,86.517,21.053,53.785
1,1x16 Conv1D 2x16 GRU-B 1x8 Dense,single,68.493,0.312,83.146,45.614,64.380
2,2x16GRU 1x16D,ensemble,58.904,0.071,79.775,26.316,53.046
3,1x16 Conv1D 2x16 GRU-B 1x8 Dense,ensemble,62.329,0.201,70.787,49.123,59.955
